In [1]:
%run 1.Functions.ipynb
%run Old_Functions.ipynb

In [2]:
path_ideal = "/Users/mahshid/Documents/Database/4Variables-poisson/Ideals/"

n = 2000
Ideals = []
for i in range(n):
    with open(path_ideal + 'Ideal{}.pickle'.format(i), 'rb') as handle:
        I = pickle.load(handle)
        Ideals.append(I)

In [6]:
def Method1(I):
    All_moves = []                                      # compute all possible moves
    for i in range(2,len(P.gens()) + 1):
        for j in range(1,i):
            All_moves.append([i,j])
    Moves = []        
    for move in All_moves:                              # Choosing a MOVE
        transformation = Transformation(I, move)        # Do the transformation on the ideal   
        Ideal = ideal(f for f in transformation)                        
        GB = Ideal.groebner_basis() 
        H = Fast_Monomial_Completion(LT_I(GB))     # Monomial Janet basis for the ideal after transfrmation
        v = Volume_Polynomial_Pommaret(Beta_Vec_P(H), X1)
        V = Coefficients(v, len(P.gens())-1)
        Moves.append([move, V])
    Moves = sorted(Moves, key=lambda x: x[1])   
        
    return Moves

In [3]:
def Method2(I):
    All_moves = []                                      # compute all possible moves
    for i in range(2,len(P.gens()) + 1):
        for j in range(1,i):
            All_moves.append([i,j])
    Moves = []        
    for move in All_moves:                              # Choosing a MOVE
        transformation = Transformation(I, move)        # Do the transformation on the ideal   
        Ideal = ideal(f for f in transformation)                        
        GB = Ideal.groebner_basis() 
        H = Fast_Monomial_Completion(LT_I(GB))     # Monomial Janet basis for the ideal after transfrmation
        v = Volume_Polynomial_Pommaret2(H, X1)
        V = Coefficients(v, len(P.gens())-1)
        Moves.append([move, V])
    Moves = sorted(Moves, key=lambda x: x[1])   
        
    return Moves

In [59]:
def Method3(I): 
    All_moves = []                                      # compute all possible moves
    for i in range(2,len(P.gens()) + 1):
        for j in range(1,i):
            All_moves.append([i,j]) 
    Moves = []        
    for move in All_moves:                              # Choosing a MOVE
        transformation = Transformation(I, move)        # Do the transformation on the ideal   
        Ideal = ideal(f for f in transformation)                        
        GB = Ideal.groebner_basis() 
        H = Fast_Monomial_Completion(LT_I(GB))     # Monomial Janet basis for the ideal after transfrmation
        B = Beta_Vector(H)
        B_P = Beta_Vec_P(H)
        D = []
        for i in range(len(P.gens())):
            D.append(abs(B[i] - B_P[i]))
        Moves.append([move, D])
    Moves = sorted(Moves, key=lambda x: x[1])   
        
    return Moves

In [74]:
def Method4(I):
    All_moves = []                                      # compute all possible moves
    for i in range(2,len(P.gens()) + 1):
        for j in range(1,i):
            All_moves.append([i,j]) 
    Moves = []        
    for move in All_moves:                              # Choosing a MOVE
        transformation = Transformation(I, move)        # Do the transformation on the ideal   
        Ideal = ideal(f for f in transformation)                        
        GB = Ideal.groebner_basis() 
        H = Fast_Monomial_Completion(LT_I(GB))     # Monomial Janet basis for the ideal after transfrmation
        B = Beta_Vector(H)
        s = Sigma(H , B)
        Moves.append([move, s])
    Moves = sorted(Moves, key=lambda x: x[1])   
        
    return Moves

In [119]:
def Method5(I):
    All_moves = []                                      # compute all possible moves
    for i in range(2,len(P.gens()) + 1):
        for j in range(1,i):
            All_moves.append([i,j]) 
    Moves = []        
    for move in All_moves:                              # Choosing a MOVE
        transformation = Transformation(I, move)        # Do the transformation on the ideal   
        Ideal = ideal(f for f in transformation)                        
        GB = Ideal.groebner_basis() 
        H = Fast_Monomial_Completion(LT_I(GB))     # Monomial Janet basis for the ideal after transfrmation
        B = Beta_Vector(H)
        s = Sigma2(H)
        Moves.append([move, s])
    Moves = sorted(Moves, key=lambda x: x[1])   
        
    return Moves

In [120]:
def Sigma(H , B):
    s = 0
    BP = Beta_Vec_P(H)
    BP.reverse()
    B.reverse()
    for k in range(len(P.gens())):
        s += (k + 2) *(B[k] - BP[k])
    return s

In [121]:
def Sigma2(H):
    s = 0
    BP = Beta_Vec_P(H)
    BP.reverse()
    for k in range(1, len(P.gens()) + 1):
        s += k * BP[k - 1]      
        
    return  round(s / Volume_Polynomial(H, max_deg(H) + 1), 3)

In [39]:
QS = []
Obstructions = []
for i in tqdm.tqdm(range(len(Ideals))):
    Q = QS_moves(Ideals[i])
    QS.append(Q)
    Obstructions.append([Q[i][0] for i in range(len(Q))])

100%|█████████████████████████████████████████| 500/500 [00:33<00:00, 14.86it/s]


In [105]:
def percentage_in_obstructions(Outputs, Obstructions, QS):
    c = 0
    s = 0
    for i in range(len(Obstructions)):
        if Outputs[i] in Obstructions[i]:
            c += 1
            s += QS[i][Obstructions[i].index(Outputs[i])][1]
    return round(c/5, 2), s        

In [129]:
QS[1]

[[[3, 2], 2]]

In [104]:
Obstructions[0].index([4,1])

2

In [103]:
A = [1,2,3,4]
A.index(1)

0

#  Method 1:
Comparing Volume polynomials (with Beta vectos of Pommaret span).

In [7]:
def Volume_Polynomial_Pommaret(B, r):
    v = 0
    B.reverse()
    for i in range(len(B)):
        v += B[i]* Combination(r + i, i)
    return v 

In [31]:
Errors1 = []    # Check increasing of Volume polynomial of Pommaret span
Outputs1 = []
for i in tqdm.tqdm(range(len(Ideals))):
    F = Ideals[i]
    Moves = Method1(F)
    Outputs1.append(Moves[-1][0])
    I = ideal(f for f in F)
    G = I.groebner_basis()
    H = Fast_Monomial_Completion(LT_I(G))
    v = Volume_Polynomial_Pommaret(Beta_Vec_P(H), X1)
    V = Coefficients(v, len(P.gens())-1)    
    #if V > Moves[-1][1]:
        #Errors1.append([i, Beta_vector(H)])
        #break

100%|█████████████████████████████████████████| 500/500 [37:00<00:00,  4.44s/it]


In [32]:
Errors

[]

In [106]:
p1 , score1 = percentage_in_obstructions(Outputs1, Obstructions, QS) 
print(p1)
print(score1)

85.6
4196


#  Method 2:
Comparing Volume polynomials (with Pommaret multiplicative variables).

In [4]:
def Volume_Polynomial_Pommaret2(H, r):
    s = 0
    for h in H:
        d = h.degree()
        s += Combination(r - d + clss(h) - 1, clss(h) - 1)
    return s

In [51]:
Errors2 = []    # Check increasing of Volume polynomial of Pommaret span
Outputs2 = []
for i in tqdm.tqdm(range(len(Ideals))):
    F = Ideals[i]
    Moves = Method2(F)
    Outputs2.append(Moves[-1][0])
    I = ideal(f for f in F)
    G = I.groebner_basis()
    H = Fast_Monomial_Completion(LT_I(G))
    v = Volume_Polynomial_Pommaret2(H, X1)
    V = Coefficients(v, len(P.gens())-1)    
    if V > Moves[-1][1]:
        Errors2.append([i, Beta_vector(H)])
        break

100%|█████████████████████████████████████████| 500/500 [14:01<00:00,  1.68s/it]


In [52]:
len(Errors2)

0

In [107]:
p2 , score2 = percentage_in_obstructions(Outputs2, Obstructions, QS) 
print(p2)
print(score2)

90.6
4510


# Method 3:
Comparing Beta vectors of Pommaret span with Beta vector of ideal
by choosing the $\beta_p$ which has the lowest vector

$(|\beta^n - \beta^n_p| , \ldots,|\beta^1 - \beta^1_p|)$

with lexicographic order.  

In [69]:
Outputs3 = []
for i in tqdm.tqdm(range(len(Ideals))):
    F = Ideals[i]
    Moves = Method3(F)
    Outputs3.append(Moves[0][0])

100%|█████████████████████████████████████████| 500/500 [36:16<00:00,  4.35s/it]


# Method 4:
Comparing Beta vectors of Pommaret span with Beta vector of ideal
by Cartan test

$\sigma = \sum_{k=1}^n (k+1) (\beta^k - \beta^k_p)$

The lowest $\sigma$ is the best. 

In [67]:
# check Positivity of score
Errors4 = []
for j in tqdm.tqdm(range(len(Ideals))):
    F = Ideals[i]
    I = ideal(f for f in F)
    G = I.groebner_basis()
    H = Fast_Monomial_Completion(LT_I(G))
    B = Beta_Vector(H)
    f = Sigma(H, B)
    if f < 0:
        Errors4.append([j, len(JB[j])]) 

100%|███████████████████████████████████████| 500/500 [1:40:15<00:00, 12.03s/it]


In [68]:
len(Errors4)

0

In [75]:
Outputs4 = []
for i in tqdm.tqdm(range(len(Ideals))):
    F = Ideals[i]
    Moves = Method4(F)
    Outputs4.append([Moves[0][0]])
    I = ideal(f for f in F)
    G = I.groebner_basis()
    H = Fast_Monomial_Completion(LT_I(G))
    B = Beta_Vector(H) 
    f = Sigma(H, B)
    if f < Moves[-1][1]:
        Errors4.append([i, f, Moves])

100%|█████████████████████████████████████████| 500/500 [54:22<00:00,  6.53s/it]


In [76]:
len(Errors4)

26

# Method 5
$\sigma = \frac{\dim_K I_{q+1}}{\sum k\beta_P^k}$

In [124]:
Errors5 = []
Outputs5 = []
for i in tqdm.tqdm(range(len(Ideals))):
    F = Ideals[i]
    Moves = Method5(F)
    Outputs5.append(Moves[-1][0])
    I = ideal(f for f in F)
    G = I.groebner_basis()
    H = Fast_Monomial_Completion(LT_I(G))
    B = Beta_Vector(H) 
    f = Sigma2(H)
    if f > Moves[-1][1]:
        Errors5.append([i, f, Moves])

100%|█████████████████████████████████████████| 500/500 [48:24<00:00,  5.81s/it]


In [125]:
len(Errors5)

0

In [126]:
p5 , score5 = percentage_in_obstructions(Outputs5, Obstructions, QS) 
print(p5)
print(score5)

88.8
5815


# Compare all methods

In [127]:
print("Method1 -> ",p1, "%    Score: ", score1)
print("Method2 -> ",p2, "%    Score: ", score2)
print("Method3 -> ",p3, "%    Score: ", score3)
print("Method4 -> ",p4, "%    Score: ", score4)
print("Method5 -> ",p5, "%    Score: ", score5)

Method1 ->  85.6 %    Score:  4196
Method2 ->  90.6 %    Score:  4510
Method3 ->  91.8 %    Score:  4475
Method4 ->  91.2 %    Score:  5641
Method5 ->  88.8 %    Score:  5815
